This notebook normalize the raster image data to 0-1 range.  

***Import necessary libraries***

In [1]:
import os
import numpy as np
import rasterio
import rasterio.features

Lets create the files variables

In [7]:
folder = "D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb"
list_bands = ['02', '03', '04', '05', '06', '07', '08', '8A', '11', '12']

Lets create the function that normalize the data based on this :  
```txt
For each band separately, normalize data to the interval [0, 1]:
```
```python
norm_data = np.clip( (data - min_val) / (max_val - min_val), 0, 1)
```
```txt
where min_val and max_val correspond respectively to the 2% and 98% percentile on the data. Any values outside the range [min_val, max_val] (eventual outliers) are clipped to 0 or 1.
```

In [9]:
def normalize_and_save_band(band, output_folder):
    with rasterio.open(band) as src:
        normalized_acquisitions = []
        for acquisition in range(1, src.count + 1):
            print(f"Acquisition {acquisition}/{src.count}")
            data_array = src.read(acquisition)
            min_val = (np.percentile(data_array, 2))
            max_val = (np.percentile(data_array, 98))
            norm_band = np.clip((data_array - min_val) / (max_val - min_val), 0, 1).astype('float32')
            print(f"Min: {min_val}, Max: {max_val}, Mean: {np.mean(norm_band)}")

            # Append the normalized acquisition to the list
            normalized_acquisitions.append(norm_band)

        # Stack the acquisitions along a new dimension (time)
        normalized_band = np.stack(normalized_acquisitions, axis=0)

        # Define the output file path for the normalized band
        output_path = os.path.join(output_folder, f'normalized_{os.path.basename(band)}.tif')

        # Create a new GeoTIFF file with a custom profile and write the normalized data
        profile = src.profile.copy()
        profile.update(count=src.count, dtype='float32')  # Set the count to the number of acquisitions
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(normalized_band)

Create the output folder.

In [10]:
output_band_folder = os.path.join(folder, 'normalized_gathered_output')
os.makedirs(output_band_folder, exist_ok=True)

Now we normalize the data and save it in the output folder.  

In [ ]:
for band in list_bands:
    print(f"Band {band}")
    band_file = os.path.join(folder, 's2_2020_B' + band + '.tif')
    normalize_and_save_band(band_file, output_band_folder)

Once done, we have our normalized data in the output folder.